# Trying to get descriptors for some molecules 
# Option 1 extract from the autoqchem database

In [1]:
import pandas as pd
#import re

import sys
sys.path.append('../')

from aqc_utils.molecule import molecule
from aqc_utils.db_functions import *
from aqc_utils.openbabel_functions import *

import hashlib
import logging
logging.basicConfig(level=logging.INFO)

In [4]:
mol_df = db_select_molecules(tags = ['JS_substrate', 'test_JulesSchleinitz'])

In [5]:
tags_coll = db_connect("tags")
mols_coll = db_connect("molecules")
log_files_coll = db_connect("log_files")

In [6]:
# helper function to create file names the same way ACQ does
def mol_fs_name(can):
    mol = input_to_OBMol(can, "string", "smi")
    return f"{mol.GetFormula()}_{hashlib.md5(can.encode()).hexdigest()[:4]}"

In [7]:
# add fs_name to the table
mol_df['file_base_name'] = mol_df['can'].map(mol_fs_name)

In [9]:
# get a cursor that iterates over the log files
cursor = log_files_coll.find({'molecule_id' :{"$in": mol_df.molecule_id.tolist()}}, {'log':1, 'can':1})

In [10]:
# path for file storage
path = '/Users/julesschleinitz/Desktop/These/Projet_stage_M1_Max/Code/SM_DFT/DFT_for_SM/log_data'

In [24]:
# dictionary to keep trach of conformer numbers
m_dict = {m:0 for m in mol_df.can.tolist()}
L = !ls {path}

for l in cursor:
    can, log = l['can'], l['log']
    fs_name = mol_fs_name(can)  
    
    if "{fs_name}_{m_dict[can]}.log" not in L:
        with open(f"{path}/{fs_name}_{m_dict[can]}.log", "w") as f:
            f.write(log)
    
        m_dict[can] += 1

In [25]:
mol_df.drop(['molecule_id', '_ids'], axis=1).to_csv(f"{path}/legend.csv")

In [32]:
all_mol_dscptrs = []
for file_name in mol_df.file_base_name.tolist():
    log = str('../log_data/' + file_name + '_0.log')
    extractor = gaussian_log_extractor(log)
    extractor.__init__
    mol_descriptors = extractor.get_descriptors()
    all_mol_dscptrs.append(mol_descriptors)

In [44]:
mol_df["descriptors_full"] = all_mol_dscptrs

In [48]:
mol_df.to_csv('../data_csv/aqc_descriptors.csv')